# PWMの値の吸い上げ

## Boardの判別

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson NANOを認識")
    I2C_BUSNUM = 1

## I2Cの認識確認

0x08: PWMの値の吸い上げ<br>
0x3c: OLED　Display<br>
0x40: PCA9685(PWMの出力)<br>
0x70: PCA9685(PWMの出力)<br>

In [ ]:
!i2cdetect -y -r $I2C_BUSNUM

## ラジコンの制御

Jetson Nano 2G, 4GはBUSNUMを1に指定します。
Jetson NXは、BUSNUMを8に指定します。

タミヤのラジコンは60Hzを指定して制御します。

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

stop = 0
try:
    with open('pwm_params.json') as f:
        json_str = json.load(f)

        stop = json_str["pwm_speed"]["stop"]
        left = json_str["pwm_steering"]["left"]
        center = json_str["pwm_steering"]["center"]
        right = json_str["pwm_steering"]["right"]
except:
    print("New setting")
        
if stop == 0:
    INITIAL_VALUE=410
else:
    INITIAL_VALUE=stop
    
SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

In [ ]:
from ipywidgets import Button, Layout, Textarea, HBox, VBox
import ipywidgets
import sys
import threading

raw_left_widget = ipywidgets.IntText(description='左')
raw_center_widget = ipywidgets.IntText(description='中央')
raw_right_widget = ipywidgets.IntText(description='右')
raw_front_widget = ipywidgets.IntText(description='前進')
raw_stop_widget = ipywidgets.IntText(description='停止')
raw_back_widget = ipywidgets.IntText(description='後進')

set_button = ipywidgets.Button(description='Set')
start_button = ipywidgets.Button(description='Connect')
stop_button = ipywidgets.Button(description='Stop')

output = ipywidgets.Output() 

TYPE_STEERING = 1
TYPE_SPEED = 2

state = 0

raw_left = 0
raw_center = 0
raw_right = 0
raw_front = 0
raw_stop = 0
raw_back = 0
running = False

def set_value(c):
    global type, running,state,raw_left,raw_center,raw_right,raw_left_widget,raw_center_widget,raw_right_widget,raw_front,raw_stop,raw_back,raw_front_widget,raw_stop_widget,raw_back_widget,execute_thread
    
    if type == TYPE_STEERING:
        if state == 0:
            raw_left = raw_left_widget.value
            state += 1
        elif state == 1:
            raw_center = raw_center_widget.value
            state += 1
        elif state == 2:
            raw_right = raw_right_widget.value
            running = False
            execute_thread.join()
            state = 0
    elif type == TYPE_SPEED:
        if state == 0:
            output.clear_output(True)
            raw_front = raw_front_widget.value
            state += 1
        elif state == 1:
            raw_stop = raw_stop_widget.value
            state += 1
        elif state == 2:
            raw_back = raw_back_widget.value
            running = False
            execute_thread.join()

def live():
    global type, running, state, raw_top_widget, raw_middle_widget, raw_bottom_widget
    i2c = smbus.SMBus(I2C_BUSNUM)
    addr = 0x08
    state = 0

    while running:
        data1 = i2c.read_i2c_block_data(addr, 0x01, 8)
        raw1 = data1[0] << 24 | data1[1] << 16 | data1[2] << 8| data1[3]
        raw2 = data1[4] << 24 | data1[5] << 16 | data1[6] << 8| data1[7]
        
        if type == TYPE_STEERING:
            if state == 0:
                raw_left_widget.value = raw1
            elif state == 1:
                raw_center_widget.value = raw1
            elif state == 2:
                raw_right_widget.value = raw1
        elif type == TYPE_SPEED:
            if state == 0:
                raw_front_widget.value = raw2
            elif state == 1:
                raw_stop_widget.value = raw2
            elif state == 2:
                raw_back_widget.value = raw2

def start(c):
    global running, execute_thread, start_time
    running = True
    execute_thread = threading.Thread(target=live)
    execute_thread.start()

def stop(c):
    global running, execute_thread
    running = False
    execute_thread.join()
    
set_button.on_click(set_value)
start_button.on_click(start)
stop_button.on_click(stop)

In [ ]:
type = TYPE_STEERING
train_eval_widget = ipywidgets.VBox([
    ipywidgets.HBox([raw_left_widget]),
    ipywidgets.HBox([raw_center_widget]),
    ipywidgets.HBox([raw_right_widget]),
    ipywidgets.HBox([start_button,set_button, stop_button])
])

display(train_eval_widget)

In [ ]:
print("raw 右: %s" % raw_left)
print("raw 中央: %s" % raw_center)
print("raw 左: %s" % raw_right)

In [ ]:
type = TYPE_SPEED
train_eval_widget = ipywidgets.VBox([
    ipywidgets.HBox([raw_front_widget]),
    ipywidgets.HBox([raw_stop_widget]),
    ipywidgets.HBox([raw_back_widget]),
    ipywidgets.HBox([start_button,set_button, stop_button])
])

display(train_eval_widget)

In [ ]:
print("raw 右: %s" % raw_front)
print("raw 中央: %s" % raw_stop)
print("raw 左: %s" % raw_back)

In [ ]:
import json

json_str = {
    "raw_steering": {
        "left": raw_left,
        "center": raw_center,
        "right": raw_right,
    },
    "raw_speed": {
        "front": raw_front,
        "stop": raw_stop,
        "back": raw_back,
    }
}

with open('raw_params.json', 'w') as f:
    json.dump(json_str, f, ensure_ascii=False)

In [ ]:
print(json_str)